In [1]:
from pprint import pprint
from copy import deepcopy
from termcolor import colored
import math

In [2]:
labirinto_draw = ''
with open('Modelos\maze01.txt', 'r') as fp:
    labirinto_draw = fp.readlines()

In [3]:
labirinto = []
for linha in labirinto_draw:
    aux = []
    for coluna in linha:
        if coluna == ' ':
            aux.append(0)
        elif coluna == 'E':
            aux.append(1)
        elif coluna == 'S':
            aux.append(2)
        else:
            aux.append(9)   
    labirinto.append(aux) 

In [4]:
class Estado:
    def __init__(self, posicao=[], custo = 0, acao = None, pai = None, custo_caminho = 0, fn=0):
        self.posicao = posicao
        self.custo = custo
        self.acao = acao
        self.pai = pai
        self.custo_caminho = custo_caminho
        self.fn = fn
    def get_custo(self):
        return self.custo
    def get_fn(self):
        return self.fn

In [5]:
def Dist_Euclidiana(x1,y1,x2,y2):
    return math.sqrt(math.pow((x2-x1),2)+math.pow((y2-y1),2))

In [6]:
def print_lista(lista):
    for i in range(len(lista)):
        #print("")
        #print("["+str(i)+"] ",end="")
        #print("Posição: ",end="")
        #print(lista[i].posicao,end="")
        #print(" - ",end="")
        #print("Custo: ",end="")
        #print(lista[i].custo)
        #print(" - ",end="")
        #print("Ação: ",end="")
        print(lista[i].acao,end="")
        #if(lista[i].pai!=None):
        #    print("Pai: ",end="")
        #    print(lista[i].pai.posicao)
        #else:
        #    print("Pai: ",end="")
        #    print(None)
        #print("Custo Caminho: ",end="")
        #print(lista[i].custo_caminho)
        #print("F(n): ",end="")
        #print(lista[i].fn)
    print("")
            

In [7]:
def Movimentos_BG(posicao_atual,saida,labirinto):
    direita=None
    esquerda=None
    cima=None
    baixo=None
    lista=[]
    
    #Direita
    if(labirinto[posicao_atual.posicao[0]][posicao_atual.posicao[1]+1]==0):
        posicao=[posicao_atual.posicao[0],posicao_atual.posicao[1]+1]
        custo=Dist_Euclidiana(posicao[0],posicao[1],saida[0],saida[1])
        acao="→"
        pai=posicao_atual
        direita=Estado(posicao,custo,acao,pai)
    #Esquerda
    if(posicao_atual.posicao[1]!=0):
        if(labirinto[posicao_atual.posicao[0]][posicao_atual.posicao[1]-1]==0):
            posicao=[posicao_atual.posicao[0],posicao_atual.posicao[1]-1]
            custo=Dist_Euclidiana(posicao[0],posicao[1],saida[0],saida[1])
            acao="←"
            pai=posicao_atual
            esquerda=Estado(posicao,custo,acao,pai)
    #Cima
    if(labirinto[posicao_atual.posicao[0]-1][posicao_atual.posicao[1]]==0):
        posicao=[posicao_atual.posicao[0]-1,posicao_atual.posicao[1]]
        custo=Dist_Euclidiana(posicao[0],posicao[1],saida[0],saida[1])
        acao="↑"
        pai=posicao_atual
        cima=Estado(posicao,custo,acao,pai)
    #Baixo
    if(labirinto[posicao_atual.posicao[0]+1][posicao_atual.posicao[1]]==0):
        posicao=[posicao_atual.posicao[0]+1,posicao_atual.posicao[1]]
        custo=Dist_Euclidiana(posicao[0],posicao[1],saida[0],saida[1])
        acao="↓"
        pai=posicao_atual
        baixo=Estado(posicao,custo,acao,pai)
    #Verificações
    if(direita!=None):
        lista.append(direita)
    if(esquerda!=None):
        lista.append(esquerda)
    if(cima!=None):
        lista.append(cima)
    if(baixo!=None):
        lista.append(baixo)
    lista=sorted(lista,key=Estado.get_custo,reverse=True)    
    return [lista.pop()]

In [8]:
def Movimentos_AE(posicao_atual,saida,labirinto,heuristica):
    direita=None
    esquerda=None
    cima=None
    baixo=None
    lista=[]
    custo_pai=posicao_atual.custo_caminho
    
    #Direita
    if(labirinto[posicao_atual.posicao[0]][posicao_atual.posicao[1]+1]==0 or labirinto[posicao_atual.posicao[0]][posicao_atual.posicao[1]+1]==2):
        posicao=[posicao_atual.posicao[0],posicao_atual.posicao[1]+1]
        custo=Dist_Euclidiana(posicao[0],posicao[1],saida[0],saida[1])
        acao="→"
        pai=posicao_atual
        custo_caminho=custo_pai+1
        fn=custo_caminho+custo
        direita=Estado(posicao,custo,acao,pai,custo_caminho,fn)
    #Esquerda
    if(posicao_atual.posicao[1]!=0):
        if(labirinto[posicao_atual.posicao[0]][posicao_atual.posicao[1]-1]==0):
            posicao=[posicao_atual.posicao[0],posicao_atual.posicao[1]-1]
            custo=Dist_Euclidiana(posicao[0],posicao[1],saida[0],saida[1])
            acao="←"
            pai=posicao_atual
            custo_caminho=custo_pai+1
            fn=custo_caminho+custo
            esquerda=Estado(posicao,custo,acao,pai,custo_caminho,fn)
    #Cima
    if(labirinto[posicao_atual.posicao[0]-1][posicao_atual.posicao[1]]==0):
        posicao=[posicao_atual.posicao[0]-1,posicao_atual.posicao[1]]
        custo=Dist_Euclidiana(posicao[0],posicao[1],saida[0],saida[1])
        acao="↑"
        pai=posicao_atual
        custo_caminho=custo_pai+1
        fn=custo_caminho+custo
        cima=Estado(posicao,custo,acao,pai,custo_caminho,fn)
    #Baixo
    if(labirinto[posicao_atual.posicao[0]+1][posicao_atual.posicao[1]]==0):
        posicao=[posicao_atual.posicao[0]+1,posicao_atual.posicao[1]]
        custo=Dist_Euclidiana(posicao[0],posicao[1],saida[0],saida[1])
        acao="↓"
        pai=posicao_atual
        custo_caminho=custo_pai+1
        fn=custo_caminho+custo
        baixo=Estado(posicao,custo,acao,pai,custo_caminho,fn)
    #Verificações
    if(direita!=None):
        lista.append(direita)
    if(esquerda!=None):
        lista.append(esquerda)
    if(cima!=None):
        lista.append(cima)
    if(baixo!=None):
        lista.append(baixo)
    #for i in range(len(lista)):
    #    if(lista[i].custo>heuristica):
    #        lista.pop(i)
    lista=sorted(lista,key=Estado.get_fn,reverse=True)    
    return lista

In [9]:
def imprimir_solucao(estado_atual,historico):
    arvore=[]
    arvore.append(estado_atual)
    historico=sorted(historico,key=Estado.get_custo)
    while(estado_atual.pai!=None):
        for i in range(len(historico)):
            if(historico[i].posicao==estado_atual.pai.posicao or estado_atual.pai.posicao==None):
                arvore.insert(0,historico[i])
                estado_atual=historico[i]
                break
    return arvore

In [10]:
def imprimir_caminho(posicao_atual,historico,labirinto_draw):
    caminho=False
    historico=imprimir_solucao(posicao_atual,historico)
    for linha in range(len(labirinto_draw)):
        for coluna in range(len(labirinto_draw[0])):
            if(coluna==len(labirinto_draw[0])):
                print(labirinto_draw[linha][coluna])
            else:
                for i in range(len(historico)):
                    if([linha,coluna]==historico[i].posicao):
                        caminho=True
                        if(historico[i].acao==None):
                            print(colored("E", 'red'),end="")
                        elif(historico[i].custo==0):
                            print(colored("S", 'red'),end="")
                        else:
                            print(colored(historico[i].acao, 'red'),end="")
                        break
                if(caminho==False):
                    print(labirinto_draw[linha][coluna],end="")
                else:
                    caminho=False               
    
    

In [11]:
def busca_gulosa(labirinto,labirinto_draw):
    entrada=[]
    saida=[]
    for linha in range(len(labirinto)):
        for coluna in range(len(labirinto[0])):
            if(labirinto[linha][coluna]==1):
                entrada.append(linha)
                entrada.append(coluna)
            if(labirinto[linha][coluna]==2):
                saida.append(linha)
                saida.append(coluna)
    fronteira=[]
    historico=[]
    novo_movimento=True
    solucao_encontrada=False
    solucao=Estado(saida,0)
    posicao_atual=Estado(entrada,Dist_Euclidiana(entrada[0],entrada[1],saida[0],saida[1]))
    historico.append(posicao_atual)#deletar
    while(solucao_encontrada != True):        
        movimentos=Movimentos_BG(posicao_atual,saida,labirinto)
        for i in range(len(movimentos)):
            for j in range(len(historico)):
                if(movimentos[i].posicao==historico[j].posicao):
                    novo_movimento=False
                    break
            if(novo_movimento==True):
                if(movimentos[i].posicao==solucao.posicao):
                    solucao_encontrada=True
                    conjunto_solucao=deepcopy(movimentos[i])
                    break
                fronteira.append(movimentos[i])
                historico.append(movimentos[i])
            novo_movimento=True
        if(solucao_encontrada!=True):
            if(fronteira != []):
                fronteira=sorted(fronteira,key=Estado.get_custo,reverse=True)
                posicao_atual=fronteira.pop()
            else:                
                imprimir_caminho(posicao_atual,historico,labirinto_draw)
                print(colored("Sem mais caminhos possiveis!",'red'))
                break
        else:
            imprimir_caminho(posicao_atual,historico,labirinto_draw)
            print(colored("Solução Encontrada",'green'))
            

In [12]:
def busca_AE(labirinto,labirinto_draw):
    entrada=[]
    saida=[]
    for linha in range(len(labirinto)):
        for coluna in range(len(labirinto[0])):
            if(labirinto[linha][coluna]==1):
                entrada.append(linha)
                entrada.append(coluna)
            if(labirinto[linha][coluna]==2):
                saida.append(linha)
                saida.append(coluna)
    fronteira=[]
    historico=[]
    novo_movimento=True
    solucao_encontrada=False
    solucao=Estado(saida,0)
    heuristica=Dist_Euclidiana(entrada[0],entrada[1],saida[0],saida[1])
    posicao_atual=Estado(entrada,heuristica,None,None,0,0)  
    historico.append(posicao_atual)
    while(solucao_encontrada != True):
        movimentos=Movimentos_AE(posicao_atual,saida,labirinto,heuristica)
        for i in range(len(movimentos)):
            for j in range(len(historico)):
                if(movimentos[i].posicao==historico[j].posicao):
                    novo_movimento=False
                    break
            if(novo_movimento==True):
                if(movimentos[i].posicao==solucao.posicao):
                    solucao_encontrada=True
                    conjunto_solucao=deepcopy(movimentos[i])
                    break
                fronteira.append(movimentos[i])
                historico.append(movimentos[i])
            novo_movimento=True
        if(solucao_encontrada!=True):
            if(fronteira != []):
                fronteira=sorted(fronteira,key=Estado.get_fn,reverse=True)
                posicao_atual=fronteira.pop()
            else:                
                imprimir_caminho(posicao_atual,historico,labirinto_draw)
                print(colored("Sem mais caminhos possiveis!",'red'))
                break
        else:
            imprimir_caminho(conjunto_solucao,historico,labirinto_draw)
            print("Custo do Caminho: ",end="")
            print(conjunto_solucao.custo_caminho)
            print(colored("Solução Encontrada",'green'))
            

In [13]:
busca_gulosa(labirinto,labirinto_draw)

+--+--+--+--+--+--+--+--+--+--+
E→→→→→|           |     |     |
+  + ↓+  +--+--+  +--+  +  +  +
|  | ↓|  |     |  |        |  |
+  + ↓+--+ ↑+  +  +  +  +--+--+
|  | ↓→→→→→→|        |  |     |
+  +--+  +--+--+--+--+  +  +  +
|     |  |        |     |  |  |
+--+  +  +  +--+  +  +--+  +  +
|  |  |     |     |     |  |  |
+  +  +--+--+  +--+--+  +--+  +
|     |        |     |        |
+--+--+  +--+--+  +  +--+  +--+
|     |  |     |  |     |     |
+--+  +  +--+  +  +--+  +--+--+
|     |  |        |  |        |
+  +--+  +  +--+--+  +--+--+  +
|        |  |        |        |
+  +  +--+  +  +--+--+  +--+--+
|  |        |                 S
+--+--+--+--+--+--+--+--+--+--+
Sem mais caminhos possiveis!


In [14]:
busca_AE(labirinto,labirinto_draw)

+--+--+--+--+--+--+--+--+--+--+
E→→→→→|           |     |     |
+  + ↓+  +--+--+  +--+  +  +  +
|  | ↓|  |     |  |        |  |
+  + ↓+--+  +  +  +  +  +--+--+
|  | ↓→→→   |        |  |     |
+  +--+ ↓+--+--+--+--+  +  +  +
|     | ↓| ↑→→→→→ |     |  |  |
+--+  + ↓+ ↑+--+↓ +  +--+  +  +
|  |  | ↓→→→| ←←↓ |     |  |  |
+  +  +--+--+ ↓+--+--+  +--+  +
|     | ←←←←←←↓| ↑→→→|        |
+--+--+ ↓+--+--+ ↑+ ↓+--+  +--+
|     | ↓|     | ↑| ↓→→→|     |
+--+  + ↓+--+  + ↑+--+ ↓+--+--+
|     | ↓| ↑→→→→→→|  | ↓→→→→→ |
+  +--+ ↓+ ↑+--+--+  +--+--+↓ +
|    ←←←↓| ↑|        | ←←←←←↓ |
+  + ↓+--+ ↑+  +--+--+ ↓+--+--+
|  | ↓→→→→→→|          ↓→→→→→→S
+--+--+--+--+--+--+--+--+--+--+
Custo do Caminho: 100
Solução Encontrada
